In [2]:
!conda install -c conda-forge beautifulsoup4 --yes 

Solving environment: ...working... done

# All requested packages already installed.



In [1]:
from urllib.request import urlopen
from urllib.request import urlretrieve
from urllib.request import Request
from bs4 import BeautifulSoup
import csv

## Download form

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = Request(url)
html = urlopen(req)
bsObj = BeautifulSoup(html,"lxml")
table = bsObj.findAll("table")[0]
rows = table.findAll("tr")

csvFile = open("C:/temp/postal_codes.csv",'a')
writer = csv.writer(csvFile)

for row in rows:
    csvRow = []
    for column in row.findAll('th'):
        csvRow.append(column.get_text())
    for cell in row.findAll('td'):
        if(cell.get_text()!=""):
            csvRow.append(cell.get_text())
    writer.writerow(csvRow)

## Import form

In [3]:
import pandas as pd

postal_data = pd.read_csv("postal_codes.csv")
postal_data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\r\n
1,M2A,Not assigned,Not assigned\r\n
2,M3A,North York,Parkwoods\r\n
3,M4A,North York,Victoria Village\r\n
4,M5A,Downtown Toronto,Harbourfront\r\n


In [4]:
postal_data.columns

Index(['Postcode', 'Borough', 'Neighbourhood\r\n'], dtype='object')

## Preprocessing table

In [5]:
postal_data.rename(columns={'Neighbourhood\r\n':'Neighbourhood'},inplace=True)

In [6]:
def trip(x):
    return x.replace('\r\n','')

In [7]:
postal_data['Neighbourhood'] = postal_data['Neighbourhood'].apply(trip)

In [8]:
postal_data.shape

(836, 3)

In [9]:
postal_data.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [11]:
not_assigned_1 = (postal_data['Borough'] != "Not assigned")  & (postal_data['Neighbourhood'] != "Not assigned")
not_assigned_2 = (postal_data['Borough'] != "Borough")  & (postal_data['Neighbourhood'] != "Neighbourhood") & (postal_data['Postcode'] != "Postcode")
data = postal_data[not_assigned_1 & not_assigned_2]

In [12]:
data.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
14,M3B,North York,Don Mills North


In [13]:
def neighborjoin(x):
    neighbours = set(x.split(','))
    return str(neighbours).replace('\'','').replace('{','').replace('}','')
        
gb = data.groupby(['Postcode','Borough']).Neighbourhood.apply(lambda x:','.join(x))

In [14]:
df = gb.reset_index()

In [15]:
df['Neighbourhood'] = df['Neighbourhood'].apply(neighborjoin)

## Final table

In [16]:
df.shape

(102, 3)

In [17]:
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"Morningside, West Hill, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview, East Birchmount Park, Kennedy Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Scarborough Village West, Cliffcrest, Cliffside"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


In [18]:
df.to_csv("postcode.csv")